<a href="https://colab.research.google.com/github/PiyushMankad/TCD-Computer-Vision/blob/master/mankadp%40tcd.ie_CV2_Question3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Question 3.a**

1. Fcn upsamples predictions by using deconvolution or backwards convolution in-network. upsampling with a factor of f is convolution with a fractional input strde of 1/f.

2. There are 3 ways the authors have improved the coarse predictions:

*   Upsampling the coarse outputs to finer predictions using interpolation.
*   patchwise training that is feeding the network with batches of random patches instead of full images for training
*   shift and stitch - stitching together a group of shifted coarse outputs to get dense predictions



In [0]:
## upload MANKADP@TCD.IE image
from google.colab import files
# cowimage = files.upload()


In [0]:
from google.colab import drive
try:
  drive.mount('/content/drive')
except:
  pass


In [0]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import models
import numpy as np
import torchvision.transforms as T
import cv2

cowimage = "../content/drive/My Drive/Colab Notebooks/Additional Files/MANKADP@TCD.IE.png"
mask_image = "../content/drive/My Drive/Colab Notebooks/Additional Files/MANKADP@TCD.IE_mask.png"
assigned_class = 10


def decode_segmap(image, nc=21):
  
  label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])

  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  
  for l in range(0, nc):
      idx = image == assigned_class
      r[idx] = label_colors[l, 0]
      g[idx] = label_colors[l, 1]
      b[idx] = label_colors[l, 2]
    
  rgb = np.stack([r, g, b], axis=2)
  return rgb


def apply_mask(im, im_pred):
    """
    Overlays the predicted class labels onto an image using the alpha channel.
    This function assumes that the background label is the black color.
    This function is provided as an inspiration for the masking function you should write.
    """
    r_channel, g_channel, b_channel = cv2.split(np.array(im_pred))
    alpha_channel = 250 * np.ones(r_channel.shape, dtype=r_channel.dtype)
    # Make background pixels fully transparent
    alpha_channel -= 250 * np.all(im_pred == np.array([0,0,0]), axis=2).astype(r_channel.dtype)
    im_pred = cv2.merge((r_channel, g_channel, b_channel, alpha_channel))
    mask = Image.fromarray(im_pred, mode='RGBA')
    masked_img = Image.fromarray(np.array(im))
    masked_img.paste(mask, box=None, mask=mask)
    return np.array(masked_img)

# define the model
fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()

# load an image
img = cv2.imread(cowimage)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
mask_img = cv2.imread(mask_image)
mask_img = cv2.cvtColor(mask_img,cv2.COLOR_BGR2RGB)
# img = Image.open(r"E:\Intelligent Systems\Comp Vision\Additional Files\MANKADP@TCD.IE.png")
plt.imshow(img); plt.show()

# transform the image
trf = T.Compose([T.ToTensor(), 
                 T.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])
inp = trf(img).unsqueeze(0)

# pass the input through the net
out = fcn(inp)['out']
print (out.shape)

# calculate labels
om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
#print(om)
print (np.unique(om))

# show segmentation output
rgb = decode_segmap(om)
plt.imshow(masked_img);plt.show()



In [0]:
rgb.shape

print("blue mask applied at cow")
masked_img = apply_mask(img,rgb)
plt.imshow(masked_img);plt.show()


In [0]:
from sklearn.metrics import jaccard_score
from google.colab.patches import cv2_imshow

mask_img = cv2.imread(mask_image)
mask_img = cv2.cvtColor(mask_img,cv2.COLOR_BGR2RGB)

## making a msak of extra classes to remove them
mask2_img = np.array(mask_img)
r,g,b = cv2.split(mask_img)
for i in range(mask_img.shape[0]):
  for j in range(mask_img.shape[1]):
    # print(mask_img[i][j])
    if not all(np.array(mask_img[i][j])) > all(np.array([0,250,0])):
      mask2_img[i][j] = [0,0,0]

## removing extra classes 
mask_img = mask_img - mask2_img
# plt.imshow(mask_img);plt.show()

## converting to gray scale for comparison
mask_img_gray = (np.array(cv2.cvtColor(mask_img,cv2.COLOR_RGB2GRAY)))#>all(np.array([0,200,0]))
rgb_gray = np.array(cv2.cvtColor(rgb,cv2.COLOR_RGB2GRAY))
# plt.imshow(mask_img_gray,cmap="gray");plt.show()
# plt.imshow(rgb_gray,cmap="gray");plt.show()

addition = np.array(mask_img_gray) + np.array(rgb_gray)

## calculating iou of these two images
plt.imshow(rgb_gray,cmap="gray");plt.show()
plt.imshow(addition,cmap="gray");plt.show()
iou = np.sum(np.divide(np.sum(rgb_gray),np.sum(addition)))

print("Intersection over Union",iou)